# A Movie Recommendation Service
### Source: https://www.codementor.io/spark/tutorial/building-a-recommender-with-apache-spark-python-example-app-part1

#### Create a SparkContext configured for local mode

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

#### File download
Small: 100,000 ratings and 2,488 tag applications applied to 8,570 movies by 706 users. Last updated 4/2015.  
Full: 21,000,000 ratings and 470,000 tag applications applied to 27,000 movies by 230,000 users. Last updated 4/2015.

In [2]:
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'

#### Download location(s)

In [3]:
import os
datasets_path = os.path.join('/home/jovyan', 'work')

small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')
complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')

#### Getting file(s)

In [4]:
import urllib.request
small_f = urllib.request.urlretrieve (small_dataset_url, small_dataset_path)
complete_f = urllib.request.urlretrieve (complete_dataset_url, complete_dataset_path)

#### Extracting file(s)

In [5]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)

## Loading and parsing datasets
Now we are ready to read in each of the files and create an RDD consisting of parsed lines. 

Each line in the ratings dataset (ratings.csv) is formatted as: 
+ userId,movieId,rating,timestamp 

Each line in the movies (movies.csv) dataset is formatted as:
+ movieId,title,genres 

The format of these files is uniform and simple, so we can use Python split() to parse their lines once they are loaded into RDDs. Parsing the movies and ratings files yields two RDDs: 
+ For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). We drop the timestamp because we do not need it for this recommender.
+ For each line in the movies dataset, we create a tuple of (MovieID, Title). We drop the genres because we do not use them for this recommender.

#### ratings.csv

In [6]:
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

# Parse
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

print ('There are {} recommendations in the complete dataset'.format(small_ratings_data.count()))
small_ratings_data.take(3)

There are 100836 recommendations in the complete dataset


[(1, 1, 4.0), (1, 3, 4.0), (1, 6, 4.0)]

#### movies.csv

In [7]:
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')
small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

# Parse
small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_titles = small_movies_data.map(lambda x: (int(x[0]),x[1]))
print ('There are {} movies in the small dataset'.format(small_movies_titles.count()))
small_movies_data.take(3)

There are 9742 movies in the small dataset


[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

## Collaborative Filtering
In Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly. 

At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has the following parameters:

+ numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
+ rank is the number of latent factors in the model.
+ iterations is the number of iterations to run.
+ lambda specifies the regularization parameter in ALS.
+ implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
+ alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

#### Selecting ALS parameters using the small dataset
In order to determine the best ALS parameters, we will use the small dataset. We need first to split it into train, validation, and test datasets.

In [8]:
# source uses see=0L, which is the previous version of python (2.x)
# 0L should be written as 0 from now on
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

#### Training phase

In [9]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ('For rank {} the RMSE is {}'.format(rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank {}'.format(best_rank))

For rank 4 the RMSE is 0.908078105265682
For rank 8 the RMSE is 0.916462973348527
For rank 12 the RMSE is 0.917665030756129
The best model was trained with rank 4


Let's have a look at how our predictions look

In [10]:
predictions.take(3)

[((372, 1084), 3.42419871162954),
 ((4, 1084), 3.866749726695713),
 ((402, 1084), 3.4099577968422152)]

Basically we have the UserID, the MovieID, and the Rating, as we have in our ratings dataset. In this case the predictions third element, the rating for that movie and user, is the predicted by our ALS model.

Then we join these with our validation data (the one that includes ratings) and the result looks as follows:

In [11]:
rates_and_preds.take(3)

[((1, 457), (5.0, 4.381060760461434)),
 ((1, 1025), (5.0, 4.705295366590298)),
 ((1, 1089), (5.0, 4.979982471805129))]

To that, we apply a squared difference and the we use the mean() action to get the MSE and apply sqrt.

Finally we test the selected model.

In [12]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is {}'.format(error))

For testing data the RMSE is 0.9113780946334407


## Using the complete dataset to build the final model

We need first to split it into training and test datasets.

In [13]:
# Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print ('There are {} recommendations in the complete dataset'.format(complete_ratings_data.count()))
complete_ratings_data.take(3)

There are 27753444 recommendations in the complete dataset


[(1, 307, 3.5), (1, 481, 3.5), (1, 1091, 1.5)]

In [14]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, \
                           iterations=iterations, lambda_=regularization_parameter)

Now we test on our testing set.

In [15]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is {}'.format(error))

For testing data the RMSE is 0.8318265262101795


## How to make recommendations
Although we aim at building an online movie recommender, now that we know how to have our recommender model ready, we can give it a try providing some movie recommendations. This will help us coding the recommending engine later on when building the web service, and will explain how to use the model in any other circumstances.

When using collaborative filtering, getting recommendations is not as simple as predicting for the new entries using a previously generated model. Instead, we need to train again the model but including the new user preferences in order to compare them with other users in the dataset. That is, the recommender needs to be trained every time we have new user ratings (although a single model can be used by multiple users of course!). This makes the process expensive, and it is one of the reasons why scalability is a problem (and Spark a solution!). Once we have our model trained, we can reuse it to obtain top recomendations for a given user or an individual rating for a particular movie. These are less costly operations than training the model itself.

So let's first load the movies complete file for later use.

In [16]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print("There are {} movies in the complete dataset".format(complete_movies_titles.count()))


There are 58098 movies in the complete dataset


Another thing we want to do, is give recommendations of movies with a certain minimum number of ratings. For that, we need to count the number of ratings per movie.

In [17]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

### Adding new user ratings
Now we need to rate some movies for the new user. We will put them in a new RDD and we will use the user ID 0, that is not assigned in the MovieLens dataset. Check the dataset movies file for ID to Tittle assignment (so you know what movies are you actually rating).

In [27]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
# User 1
# new_user_ratings = [
#      (0,6872,2), # House of the Dead, The (2003)
#      (0,356,4), # Forrest Gump (1994)
#      (0,72998,5), # Avatar (2009)
#      (0,184053,2), # Battle Planet (2008)
#      (0,73017,5), # Sherlock Holmes (2009)
#      (0,170827,4), # The Mummy (2017)
#      (0,137857,3), # The Jungle Book (2016)
#      (0,63992,4), # Twilight (2008)
#      (0,121231,1), # It Follows (2014)
#      (0,7842,5) # Dune (2000)
#     ]

# User 2
new_user_ratings = [
     (0,74458,3), # Shutter Island (2010)
     (0,377,1), # Speed (1994)
     (0,480,2), # Jurassic Park (1993)
     (0,588,4), # Aladdin (1992)
     (0,1370,2), # Die Hard 2 (1990)
     (0,2273,3), # Rush Hour (1998)
     (0,2985,5), # RoboCop (1987)
     (0,89745,4), # Avengers, The (2012)
     (0,100498,4), # Good Day to Die Hard, A (2013)
     (0,104913,5) # Rush (2013)
    ]

new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: {}'.format(new_user_ratings_RDD.take(10)))

New user ratings: [(0, 74458, 3), (0, 377, 1), (0, 480, 2), (0, 588, 4), (0, 1370, 2), (0, 2273, 3), (0, 2985, 5), (0, 89745, 4), (0, 100498, 4), (0, 104913, 5)]


Now we add them to the data we will use to train our recommender model. We use Spark's union() transformation for this.

In [28]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

And finally we train the ALS model using all the parameters we selected before (when using the small dataset).

In [29]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed,
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ('New model trained in {} seconds'.format(round(tt,3)))

New model trained in 130.371 seconds


## Getting top recommendations
Let's now get some recommendations! For that we will get an RDD with all the movies the new user hasn't rated yet. We will them together with the model to predict ratings.

In [30]:
# new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # original version: get just movie IDs
new_user_ratings_ids = list(map(lambda x: x[1], new_user_ratings)) # fixed version: get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

We have our recommendations ready. Now we can print out the 25 movies with the highest predicted ratings. And join them with the movies RDD to get the titles, and ratings count in order to get movies with a minimum number of counts. First we will do the join and see what does the result looks like.

In [31]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(6216,
  ((3.1354341280252385, 'Nowhere in Africa (Nirgendwo in Afrika) (2001)'),
   717)),
 (124320, ((3.043077982733017, 'Once a Thief (1965)'), 1)),
 (83916, ((3.3906833843789723, 'Blues in the Night (1941)'), 9))]

So we need to flat this down a bit in order to have (Title, Rating, Ratings Count).

In [32]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

### Scenario 1
Finally, get the highest rated recommendations for the new user, filtering out movies with less than 25 ratings.

In [33]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(15, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n{}'.format('\n'.join(map(str, top_movies))))

TOP recommended movies (with more than 25 reviews):
('Cosmos', 4.304052591770052, 157)
('Black Mirror', 4.2800071923350185, 180)
('Fight Club (1999)', 4.268904222349189, 65678)
('Black Mirror: White Christmas (2014)', 4.245457328188941, 1074)
('Iqbal (2005)', 4.195046557302881, 25)
('Demetri Martin. Person. (2007)', 4.188506366690313, 26)
('In This Corner of the World (2016)', 4.187533683570239, 34)
('Planet Earth II (2016)', 4.172301970226062, 853)
('Band of Brothers (2001)', 4.172168619987534, 984)
('Death Note: R2 - L o Tsugu Mono (2008)', 4.1685806888083565, 37)
('Pulp Fiction (1994)', 4.166725839117831, 92406)
('Tom Segura: Completely Normal (2014)', 4.161717851714196, 33)
('Death Note Rewrite: Genshisuru Kami (2007)', 4.159260014924698, 65)
('Ghost in the Shell: Stand Alone Complex - The Laughing Man (2005)', 4.1470897773242505, 193)
('Horace and Pete (2016)', 4.12612686729198, 36)


### Scenario 2
Finally, get the highest rated recommendations for the new user, filtering out movies with less than 100 ratings.

In [34]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=100).takeOrdered(15, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 100 reviews):\n{}'.format('\n'.join(map(str, top_movies))))

TOP recommended movies (with more than 100 reviews):
('Cosmos', 4.304052591770052, 157)
('Black Mirror', 4.2800071923350185, 180)
('Fight Club (1999)', 4.268904222349189, 65678)
('Black Mirror: White Christmas (2014)', 4.245457328188941, 1074)
('Planet Earth II (2016)', 4.172301970226062, 853)
('Band of Brothers (2001)', 4.172168619987534, 984)
('Pulp Fiction (1994)', 4.166725839117831, 92406)
('Ghost in the Shell: Stand Alone Complex - The Laughing Man (2005)', 4.1470897773242505, 193)
('Berserk: The Golden Age Arc 2 - The Battle for Doldrey (2012)', 4.1200005967698505, 104)
('"Dark Knight', 4.0727470557543235, 44741)
('"Matrix', 4.070637033276089, 84545)
('Planet Earth (2006)', 4.04852435546913, 1384)
('Inception (2010)', 4.048320612436465, 41475)
('Generation Kill (2008)', 4.043645533892288, 449)
('Interstellar (2014)', 4.040430969279091, 23081)


## Getting individual ratings
Another useful usecase is getting the predicted rating for a particular movie for a given user. The process is similar to the previous retreival of top recommendations but, instead of using predcitAll with every single movie the user hasn't rated yet, we will just pass the method a single entry with the movie we want to predict the rating for.

In [26]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
# individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD = new_ratings_model.predictAll(my_movie)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=500, rating=3.6825871397617913)]

## Highest rated recommendations for the User 1

### Scenario 1: Filtering out movies with less than 25 ratings  
TOP recommended movies (with more than 25 reviews):   
('Loose Change 9/11: An American Coup (2009)', 5.0937540352535216, 46)  
('Endless Love (2014)', 4.955972863573294, 139)  
("Naomi and Ely's No Kiss List (2015)", 4.931225123912871, 49)  
('Love and Honor (2012)', 4.913356024067662, 35)  
('Bad Boys 3', 4.888402787838474, 36)  
("Tyler Perry's Meet the Browns (2008)", 4.870898201889149, 26)  
('Aashiqui 2 (2013)', 4.841636983245734, 33)  
('Cats (1998)', 4.83495538112009, 25)  
('Woodlawn (2015)', 4.797499752806056, 25)  
('"Burma Conspiracy', 4.793165542949888, 25)  
('My Little Pony: The Movie (2017)', 4.771808804793227, 30)  
('"Cutting Edge: Going for the Gold', 4.749200956864247, 29)  
('Miracles from Heaven (2016)', 4.7406111902338885, 60)  
("2016: Obama's America (2012)", 4.7387159056584, 35)  
('Now You See Me 2 (2016)', 4.737506853460203, 2334)  

### Scenario 2: Filtering out movies with less than 100 ratings  
TOP recommended movies (with more than 100 reviews):   
('Endless Love (2014)', 4.955972863573294, 139)  
('Now You See Me 2 (2016)', 4.737506853460203, 2334)  
('Now You See Me (2013)', 4.723650984128929, 6361)  
('"Fuck You', 4.717321694139565, 116)  
('The Choice (2016)', 4.714798839950096, 100)  
('"Love', 4.713586631963038, 576)  
('The Longest Ride (2015)', 4.704743974659364, 501)  
('Avengers: Infinity War - Part I (2018)', 4.687435350747034, 2668)  
('Act of Valor (2012)', 4.684100684176878, 390)  
('The Best of Me (2014)', 4.679041213429421, 260)  
('Safe Haven (2013)', 4.677735559633505, 619)  
('Doctor Who: Voyage Of The Damned (2007)', 4.671443335800988, 224)  
('"Last Song', 4.630565823950083, 489)  
('"Lucky One', 4.624194909946811, 467)  
('Life as We Know It (2010)', 4.622111647139835, 1116)  

## Highest rated recommendations for the User 2

### Scenario 1: Filtering out movies with less than 25 ratings    
('Cosmos', 4.304052591770052, 157)  
('Black Mirror', 4.2800071923350185, 180)  
('Fight Club (1999)', 4.268904222349189, 65678)  
('Black Mirror: White Christmas (2014)', 4.245457328188941, 1074)  
('Iqbal (2005)', 4.195046557302881, 25)  
('Demetri Martin. Person. (2007)', 4.188506366690313, 26)  
('In This Corner of the World (2016)', 4.187533683570239, 34)  
('Planet Earth II (2016)', 4.172301970226062, 853)  
('Band of Brothers (2001)', 4.172168619987534, 984)  
('Death Note: R2 - L o Tsugu Mono (2008)', 4.1685806888083565, 37)  
('Pulp Fiction (1994)', 4.166725839117831, 92406)  
('Tom Segura: Completely Normal (2014)', 4.161717851714196, 33)  
('Death Note Rewrite: Genshisuru Kami (2007)', 4.159260014924698, 65)  
('Ghost in the Shell: Stand Alone Complex - The Laughing Man (2005)', 4.1470897773242505, 193)  
('Horace and Pete (2016)', 4.12612686729198, 36)  

### Scenario 2: Filtering out movies with less than 100 ratings    
TOP recommended movies (with more than 100 reviews):  
('Cosmos', 4.304052591770052, 157)  
('Black Mirror', 4.2800071923350185, 180)  
('Fight Club (1999)', 4.268904222349189, 65678)  
('Black Mirror: White Christmas (2014)', 4.245457328188941, 1074)  
('Planet Earth II (2016)', 4.172301970226062, 853)  
('Band of Brothers (2001)', 4.172168619987534, 984)  
('Pulp Fiction (1994)', 4.166725839117831, 92406)  
('Ghost in the Shell: Stand Alone Complex - The Laughing Man (2005)', 4.1470897773242505, 193)  
('Berserk: The Golden Age Arc 2 - The Battle for Doldrey (2012)', 4.1200005967698505, 104)  
('"Dark Knight', 4.0727470557543235, 44741)  
('"Matrix', 4.070637033276089, 84545)  
('Planet Earth (2006)', 4.04852435546913, 1384)  
('Inception (2010)', 4.048320612436465, 41475)  
('Generation Kill (2008)', 4.043645533892288, 449)  
('Interstellar (2014)', 4.040430969279091, 23081)  